### 3. This question relates to the task of Digital Signature Verification using RSA. Please,follow the underlying steps:


a. Generate RSA key pairs, where the public key encryption exponent “e” should be 3. And consider the message “m” to be your roll number (all small letters). For eg- "p21cs011".


In [1]:
from sympy import isprime
import random
def randomPrimeGen(n):
    p=""
    for i in range(3*n*n):
        p="1"
        # random string of 1s and 0s of length n-1
        p+=format(random.getrandbits(n-1),'0b')
        if(isprime(int(p,2))):
            return p
    return False
def GenModulus(n):
    p=randomPrimeGen(n)
    q=randomPrimeGen(n)
    while(p==q):
        q=randomPrimeGen(n)
    return int(p,2)*int(q,2),int(p,2),int(q,2)



In [2]:
from sympy import gcd
import random
def GenRSA(n):
    N,p,q=GenModulus(n)
    phi=(p-1)*(q-1)

    e=3 #GIVEN
    g = gcd(e, phi)
    while g != 1:
        # e = random.randrange(1, phi)
        
        N,p,q=GenModulus(n)
        phi=(p-1)*(q-1)
        g = gcd(e, phi)
    
    d = pow(e, -1, phi)
    print("phi=",phi)
    return N,e,d

N,e,d=GenRSA(128)
print("The output in base10 is:")
print("N=",N)
print("e=",e)
print("d=",d)

        

phi= 100321844372279822542369415622282017544915220159319753714424541786264450546936
The output in base10 is:
N= 100321844372279822542369415622282017545550193376401076566869457415468205707027
e= 3
d= 66881229581519881694912943748188011696610146772879835809616361190842967031291


b. Digitally sign the message “m” using the private key with PKCS1.5 padding to
obtain signed text “x” (Inbuilt libraries support automatic message conversion
followed by encryption). You can use any cryptographic hashing function for the
same.
Note - Private key can be used only once for signing the original message.


In [3]:
import hashlib
import os
#here key_size is taken in BYTES (not bits)(32 bytes=256 bits)
def pkcs1_v1_5_pad(message, key_size):

    hash = hashlib.sha1(message).digest()


    # Create the ASN.1 prefix for SHA-1 add only lenght included here
    asn1 = len(hash)
    asn1=asn1.to_bytes(asn1.bit_length()//8+1, byteorder='big')

    
    # Calculate the number of padding bytes needed
    padding_length = key_size - len(asn1) - len(hash) - 3

  
    padding = b'\xff' * padding_length

 
    padded_message = b'\x00\x02' + padding + b'\x00' + asn1 + hash 
   

    return padded_message

M='b21cs50'
M=M.encode()
print("the padded message is:",pkcs1_v1_5_pad(M,32))

the padded message is: b"\x00\x02\xff\xff\xff\xff\xff\xff\xff\xff\x00\x14|\xc9\xeb;\x9e\x97\xb0\x8e\xe9$\x03\xc6'\xd4\x7f\xb4\x1bR\xe1`"


# Adding padding and signing

In [4]:
import base64
def pad_and_sign(M,private_key):
    M=M.encode()
    M_bytes=pkcs1_v1_5_pad(M, 32)
    M=int.from_bytes(M_bytes, byteorder='big')

    N,d=private_key
    signed_decimal=pow(M,d,N)



    signed_bytes=signed_decimal.to_bytes((signed_decimal.bit_length() + 7) // 8, 'big')

    signed_base64=base64.b64encode(signed_bytes)
    return signed_base64

M='b21cs050'

signed=pad_and_sign(M,(N,d))
print("The signed message is:")
print(signed)



The signed message is:
b'rormAurbiAkwaRiEjO8pDf9PSOeJxeVxuNTYARzQ6D8='


c. Design a server (or function) that accepts the ciphertext, authenticates the sender
by decrypting the signed message using the public key and verifying the message
(plaintext) as per preliminaries - “Message Verification”. The server returns a
boolean value indicating the verification results. For instance - If the format is not
valid, the server responds with “False” value. On the other hand, if the format is
valid, the server returns “True”.
That is, there exists a PKCS Padding oracle to tell you if the padding of a message is valid or not.


In [5]:
def decrypt(signed_base64,public_key):
    signed_bytes=base64.b64decode(signed_base64)
    signed_decimal=int.from_bytes(signed_bytes, 'big')
    
    N,e=public_key
    M=pow(signed_decimal,e,N)

    M_bytes=M.to_bytes((M.bit_length() + 7) // 8, 'big')
    if(len(M_bytes)<32):
        M_bytes=bytes(32-len(M_bytes))+M_bytes
    return M_bytes


def pkcs1_v1_5_verify(padded_message, key_size):
    # Check the initial bytes
    if padded_message[:2] != b'\x00\x02':
        return False

    separator_index = padded_message.find(b'\x00', 2)

    # Check if the separator was found and if it's in the correct position
    if separator_index < 0 :
        try:
            asn1=padded_message[separator_index+1]
            hash=padded_message[separator_index+2:]
            if(len(hash)!=asn1.to_bytes(asn1.bit_length()//8+1, byteorder='big')):
                return False
        except:
            return False
        return False

    return True


In [6]:
def padding_oracle(Cyphertext):
    
    decrypted=decrypt(Cyphertext,(N,e))[:32]

    veri=pkcs1_v1_5_verify(decrypted, 32)
    return veri

print("The result of the padding oracle on original signed message is:")

result=padding_oracle(signed)   
print(result)

The result of the padding oracle on original signed message is:
True


Your task is to generate another ciphertext that passes the PKCS padding oracle
verification. Importantly, the direct use of both public & private keys is prohibited. The
task should be accomplished using only the PKCS padding oracle

In [7]:
def base64_to_decimal_multiply_bytes(signed_base64,num):
    signed_bytes=base64.b64decode(signed_base64)
    signed_decimal=int.from_bytes(signed_bytes, 'big')
    signed_decimal=((signed_decimal%N)*(num%N))%N
    signed_bytes=signed_decimal.to_bytes((signed_decimal.bit_length() + 7) // 8, 'big')
    signed_base64=base64.b64encode(signed_bytes)
    return signed_base64


In [8]:
# multiply the cyphertext by random number
M='b21cs050'
Signedmessage=pad_and_sign(M,(N,d))


num=2
other_signed_message=base64_to_decimal_multiply_bytes(Signedmessage,num)
while(padding_oracle(other_signed_message)==False):
    num+=1
    other_signed_message=base64_to_decimal_multiply_bytes(Signedmessage,num)
print("The other cyphertext which passes the padding oracle is:",other_signed_message)


The other cyphertext which passes the padding oracle is: b'xqKNEQWXh3dnDjSPOhFIVIoLIp1lK9F9B1x08lLL1+4='


In [9]:
# actual signed message
print("The actual signed message is:",Signedmessage)

#the other signed message is 
print("The other signed message is:",other_signed_message)

if(Signedmessage!=other_signed_message):
    print("The other signed message is different from the actual signed message")
   

The actual signed message is: b'rormAurbiAkwaRiEjO8pDf9PSOeJxeVxuNTYARzQ6D8='
The other signed message is: b'xqKNEQWXh3dnDjSPOhFIVIoLIp1lK9F9B1x08lLL1+4='
The other signed message is different from the actual signed message


In [10]:
# decryption of the other cyphertext
print("The decryption of the other signed message is",decrypt(other_signed_message,(N,e)).hex())

The decryption of the other signed message is 00027283f03ec88dc296df770035218e31f136beed05e224c4dff845b86a9388
